In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_rows', 500)

In [ ]:
# root_dir = r"E:\Nextstep\MLChallenge\DeHaDo_AI"
root_dir = r"F:\Muthu_2023\Personal\NextStep\Hackathon\DeHaDo_AI"
file = 'Inference_test_trocr_large.csv'

In [ ]:
# model_res_path = root_dir + "/" + r"tokenwise-dehado-ai\results\OCR_Results"
model_res_path = root_dir + "/" + r"tokenwise-dehado-ai\results\TrOCR_Results"
df_pred = pd.read_csv(model_res_path + "/" + file)

In [ ]:
label_file = root_dir + "/" + r"results\OCR_Results\test.csv"
df_act = pd.read_csv(label_file)

# Preprocessing

In [ ]:
df_act['folder'] = df_act['image'].apply(lambda x: x.split("/")[2])
df_act['file'] = df_act['image'].apply(lambda x: x.split("/")[-1].replace(".jpg", ""))
df_act.drop('image', axis=1, inplace=True)
df_act.head()

In [ ]:
df_pred.drop('Unnamed: 0', axis=1, inplace=True)

In [160]:
df_pred.rename(columns={'text': 'pred_text'}, inplace=True)
df_pred.head()

,folder,file,pred_text
0,MIT_290,nationality,Indian .
1,MIT_290,Dateofbirth,3/6 ( 1972 .
2,MIT_290,Fatherhusbandname,Bahadurjit Aggarwal
3,MIT_290,permanentaddress,"H.NO. 18 , LOYAL CURCLE , JALANDHAR-309"
4,MIT_290,experience,7 . YEARS AT TALWAR-SHA .


# Combine Label and Predictions

In [ ]:
df_comb = pd.merge(df_pred, df_act, on = ['folder', 'file'], how='left')

In [ ]:
df_comb.rename(columns={'text': 'actual_text'}, inplace=True)
df_comb.head()


In [ ]:
df_comb.to_csv('Combined_Inference_Large.csv')

# EDA on Labels

- Special character distibution by field
- Distribution of special characters across field

In [ ]:
d_special = {}
def count_special_characters(data):
    cnt = 0
    for char in data:
        if char in string.punctuation:
            cnt += 1
            if char in d_special:
                d_special[char] += 1
            else:
                d_special[char] = 1
    return cnt            

In [ ]:
df_comb['n_special'] = df_comb['actual_text'].apply(count_special_characters)

In [ ]:
res = df_comb.groupby('file')['n_special'].sum()
res[res>0].sort_values(ascending=False).plot(kind='bar')

In [ ]:
d_special

### Inference:
- Only 6 different special characters are present with '&' being neglegible

In [ ]:
def count_special_character(data, s_chr):
    cnt = 0
    if 
    for char in data:
        if char in string.punctuation:
            cnt += 1
            if char in d_special:
                d_special[char] += 1
            else:
                d_special[char] = 1
    return cnt   

In [ ]:
df_comb['n_hipen'] = df_comb['actual_text'].apply(lambda x: x.count('-'))
df_comb['n_dot'] = df_comb['actual_text'].apply(lambda x: x.count('.'))
df_comb['n_comma'] = df_comb['actual_text'].apply(lambda x: x.count(','))
df_comb['n_plus'] = df_comb['actual_text'].apply(lambda x: x.count('+'))
df_comb['n_slash'] = df_comb['actual_text'].apply(lambda x: x.count('/'))
df_comb.head()

In [ ]:
df_comb[df_comb['n_special'] > 0].groupby('file').sum()

# Compute Metrics

In [ ]:
df_comb_base = pd.read_csv(r"E:\Nextstep\MLChallenge\DeHaDo_AI\Results\OCR\Combined_Inference_Base.csv")
df_comb_base.head()

In [ ]:
compute_cer_wer(df_comb_base['pred_text'], df_comb_base['actual_text'])

In [ ]:
print("CER: ", char_error_rate(df_comb_base['actual_text'], df_comb_base['pred_text']))
print("WER: ", word_error_rate(df_comb_base['actual_text'], df_comb_base['pred_text']))

In [ ]:
import time
import psutil
import numpy as np

# --- METRICS & EFFICIENCY ---
def levenshtein_distance(s1, s2):
    if isinstance(s1, str):
        s1 = list(s1)
    if isinstance(s2, str):
        s2 = list(s2)
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

def word_error_rate(y_true, y_pred):
    total_words = 0
    total_errors = 0
    for t, p in zip(y_true, y_pred):
        t_words = t.split()
        p_words = p.split()
        total_words += len(t_words)
        total_errors += levenshtein_distance(t_words, p_words)
    return total_errors / total_words if total_words > 0 else 0.0

def char_error_rate(y_true, y_pred):
    total_chars = 0
    total_errors = 0
    for t, p in zip(y_true, y_pred):
        total_chars += len(t)
        total_errors += levenshtein_distance(t, p)
    return total_errors / total_chars if total_chars > 0 else 0.0

def field_accuracy(y_true, y_pred):
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_true, y_pred)

def document_level_accuracy(y_true, y_pred, doc_ids):
    from collections import defaultdict
    doc_true = defaultdict(list)
    doc_pred = defaultdict(list)
    for doc_id, t, p in zip(doc_ids, y_true, y_pred):
        doc_true[doc_id].append(t)
        doc_pred[doc_id].append(p)
    correct_docs = 0
    for doc_id in doc_true:
        if doc_true[doc_id] == doc_pred[doc_id]:
            correct_docs += 1
    return correct_docs / len(doc_true) if doc_true else 0.0


In [ ]:
char_error_rate(df_comb[df_comb['file'] == 'Dateofbirth']['actual_text'], df_comb[df_comb['file'] == 'Dateofbirth']['pred_text'])

In [ ]:
field_list = df_comb['file'].unique()
cer = {}
wer = {}
for field in field_list:
    # print(field)
    predictions = df_comb[df_comb['file'] == field]['pred_text']
    actual = df_comb[df_comb['file'] == field]['actual_text']
    cer[field] = char_error_rate(actual, predictions)
    wer[field] = word_error_rate(actual, predictions)

In [ ]:
# import seaborn as sns
# sns.barplot(x = cer.keys(), y = cer.values())
# plt.xticks(rotation=90)
# plt.show()

In [ ]:
metrics_large = pd.DataFrame(cer.values(), columns=['cer'], index = cer.keys())
metrics_large['wer'] = wer
metrics_large

In [ ]:
df_processed = df_comb.copy()
df_processed['processed_text'] = df_processed['pred_text']

In [ ]:
def remove_special_char(text):
    cleaned_text = re.sub(r'[^A-Za-z0-9]+', '', text)
    return cleaned_text

In [ ]:
def numeric_only(text):
    cleaned_text = re.sub(r'[^0-9]+', '', text)
    return cleaned_text

In [ ]:
def char_space(text):
    cleaned_text = re.sub(r'[^A-Za-z ]+', '', text)
    return cleaned_text.strip()

In [207]:
def lang_specific(text, camelcase=True):
    text = re.sub(r'[^A-Za-z ,]+', '', text)
    text = re.sub(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', text) # Remove special character at start and end
    text = re.sub(r'[^A-Za-z0-9, ]+', ',', text) # Remove single special character to comma
    text = re.sub(r'\s+,', ',', text) # Remove extra spaces before comma
    words = []
    for word in text.split():
        if camelcase:
            capital_letters = sum(1 for char in text if char.isupper())
            if capital_letters <= len(word)//2:
                # word = word.strip().lower()
                word = word.capitalize().strip()
            else:
                word = word.strip()
        else:
            word = word.strip()
        word = autocorrect_words(word, field)
        words.append(word)
    return " ".join(words)

In [ ]:
def date_specific(text):
    text = text.replace('(', '/').replace(')', '/') # Slash is mis represented as ( by model
    cleaned_text = re.sub(r'[^0-9\-/]+', '', text)
    return cleaned_text

In [ ]:
def bloodgroup_specific(text):
    cleaned_text = re.sub(r'[^ABOTabot0-1+\-]+', '', text)
    cleaned_text = cleaned_text.replace('t', '+').replace('0', 'O')
    return cleaned_text.upper().strip()

In [ ]:
def generic(text):
    cleaned_text = re.sub(r'[^A-Za-z0-9 +\-]+', '', text)
    return cleaned_text.strip()

In [ ]:
def reference_specific(text):
    cleaned_text = re.sub(r'[^A-Za-z0-9 \-]+', '', text)
    words = cleaned_text.split("-")
    if len(words) == 1:        
        cleaned_text = " - ".join([re.sub(r'[^A-Za-z]+', '', text), re.sub(r'[^0-9]+', '', text)])
    elif len(words) == 2:
        cleaned_text = " - ".join([process_words2(words[0]).strip(), re.sub(r'[^0-9]+', '', words[1])])
        
    # for i in range(len(words)):
    #     if i+1 <= len(words):
    #         check_words()
    return cleaned_text.strip()

In [210]:
with open("field_vocabulary.json", "r") as f:
    field_vocab = json.load(f)

In [216]:
def autocorrect_words(text, field):
    if field in field_vocab:
        training_labels = field_vocab[field]
        dist = [levenshtein_distance(text, ele) for ele in training_labels]
        
        min_dist, min_dist_ind = np.min(dist), np.argmin(dist)
        corrected = training_labels[min_dist_ind]
        if ((min_dist <= 1) or ((min_dist <= 2 and len(corrected) > 4))) and dist.count(min_dist) <= 1:
            return corrected
        else:
            return text
    else:
        return text

In [ ]:
def process_words2(text):
    words = []
    for word in text.split():
        word = word.capitalize().strip()
        word = autocorrect_words(word, field)
        words.append(word)
    return " ".join(words)

In [ ]:
def alphabets_only(text):
    cleaned_text = re.sub(r'[^A-Za-z]+', '', text)
    return cleaned_text.capitalize()

In [ ]:
def process_words(text, camelcase=False):
    words = []
    for word in text.split():
        if camelcase:
            capital_letters = sum(1 for char in text if char.isupper())
            if capital_letters <= len(word)//2:
                # word = word.strip().lower()
                word = word.capitalize()
            else:
                word = word.strip()
        else:
            word = word.strip()
        words.append(word)
    return " ".join(words)

In [ ]:
def alphanum_specialchar(text):
    cleaned_text = re.sub(r'[^A-Za-z0-9 +\-/,.]+', '', text)
    return cleaned_text.strip()

In [ ]:
def address_specific(text):
    text = re.sub(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', text) # Remove special character at start and end
    text = re.sub(r'[^A-Za-z0-9, ]+', ',', text) # Remove single special character to comma
    text = re.sub(r'\s+,', ',', text) # Remove extra spaces before comma
    return text

In [103]:
alphatonum = {"o": "0", "O": "0", "b": "6", "z": "2", "Z": "2", "B": "8", "G": "6", "I": "1", "T":"7"}
numtoalpha = {val: key for key, val in alphatonum.items()}
schartonum = {"/": "1", "\\": "1", "(": "1"}


def alpha2num(text):
    text_transform = ""
    for char in text:
        if char.isalpha():
            if char in alphatonum:
                text_transform += alphatonum[char]
            else:
                text_transform += char
        else:
            text_transform += char
    return text_transform

def num2alpha(text):
    text_transform = ""
    for char in text:
        if char.isdigit():
            if char in numtoalpha:
                text_transform += numtoalpha[char]
            else:
                text_transform += char
        else:
            text_transform += char
    return text_transform

def special2num(text):
    text_transform = ""
    for char in text:
        if not char.isalnum():
            if char in schartonum:
                text_transform += schartonum[char]
            else:
                text_transform += char
        else:
            text_transform += char
    return text_transform

In [78]:
def pan_specific(text):
    text = remove_special_char(text)
    text_transform = ""
    if len(text) == 10:
        text_transform = num2alpha(text[:5])
        text_transform += alpha2num(text[5:-1])
        text_transform += num2alpha(text[-1])
    else:
        text_transform = text
    
    return text_transform.upper()

In [213]:
def post_processing(data):
    if data[0] in ['Dateofbirth', 'date']:
        return date_specific(data[1])
    elif data[0] in ['nationality', 'gender', 'maritalstatus']:
        clean_text = alphabets_only(data[1])
        return autocorrect_words(clean_text, data[0])    
    elif data[0] in ['candidatename', 'Fatherhusbandname', 'place']:
        clean_text = char_space(data[1])
        return process_words(clean_text, True)
    elif data[0] in ['contactnumber', 'AlternateNo', 'aadhaarcard']:
        clean_text = special2num(alpha2num(data[1]))
        return numeric_only(clean_text)
    elif data[0] in ['permanentaddress', 'presentaddress']:
        clean_text = alphanum_specialchar(data[1])
        clean_text = address_specific(clean_text)
        return process_words(clean_text, False)
    elif data[0] in ['bloodgroup']:
        return bloodgroup_specific(data[1])
    elif data[0] in ['experience', 'experience1']:
        clean_text = generic(data[1])
        return process_words(clean_text, False)
    elif data[0] in ['qualification']:
        clean_text = generic(data[1])
        clean_text = process_words(clean_text, True)        
        return autocorrect_words(clean_text, data[0]) 
    elif data[0] in ['referencescmob1', 'referencescmob2']:
        clean_text = reference_specific(data[1])
        return clean_text
    elif data[0] in ['pancard']:
        return pan_specific(data[1])
    elif data[0] in ['languageknown']:
        return lang_specific(data[1])
    else:
        return ""

In [214]:
df_processed_backup = df_processed.copy()

In [217]:
df_processed['processed_text'] = df_processed[['file', 'pred_text']].apply(post_processing, axis=1)


In [218]:
char_error_rate(df_processed['actual_text'], df_processed['processed_text'])

0.15770352518870723

In [219]:
word_error_rate(df_processed['actual_text'], df_processed['processed_text'])

0.44080110893360736

In [220]:
df_processed['field_acc_pred'] = df_processed['pred_text'] == df_processed['actual_text']
df_processed['field_acc_processed'] = df_processed['processed_text'] == df_processed['actual_text']

# Metrics

- CER and WER between predicted and processed texts

In [221]:
def final_score(data):
    return ((100 - (data[0] * 100)) * 0.35) + \
           ((100 - (data[1] * 100)) * 0.35) + \
           (data[2] * 0.15)

In [222]:
cer = {}
wer = {}
f_acc = {}
for field in field_list:
    df_temp = df_processed[df_processed['file']==field]
    cer[field] = [char_error_rate(df_temp['actual_text'], df_temp['pred_text']), char_error_rate(df_temp['actual_text'], df_temp['processed_text'])]
    wer[field] = [word_error_rate(df_temp['actual_text'], df_temp['pred_text']), word_error_rate(df_temp['actual_text'], df_temp['processed_text'])]
    f_acc[field] = [field_accuracy(df_temp['actual_text'], df_temp['pred_text']), field_accuracy(df_temp['actual_text'], df_temp['processed_text'])]
cer['All'] = [char_error_rate(df_processed['actual_text'], df_processed['pred_text']), char_error_rate(df_processed['actual_text'], df_processed['processed_text'])]
wer['All'] = [word_error_rate(df_processed['actual_text'], df_processed['pred_text']), word_error_rate(df_processed['actual_text'], df_processed['processed_text'])]
f_acc['All'] = [field_accuracy(df_processed['actual_text'], df_processed['pred_text']), field_accuracy(df_processed['actual_text'], df_processed['processed_text'])]

cer_df = pd.DataFrame.from_dict(cer, orient='index', columns = ['cer_pred', 'cer_processed'])
wer_df = pd.DataFrame.from_dict(wer, orient='index', columns = ['wer_pred', 'wer_processed'])
facc_df = pd.DataFrame.from_dict(f_acc, orient='index', columns = ['facc_pred', 'facc_processed'])
metrics_df = pd.concat([cer_df, wer_df, facc_df], axis=1)

metrics_df['final_score_pred'] = metrics_df[['cer_pred', 'wer_pred', 'facc_pred']].apply(final_score, axis=1)
metrics_df['final_score_processed'] = metrics_df[['cer_processed', 'wer_processed', 'facc_processed']].apply(final_score, axis=1)                                                                                     
metrics_df    

,cer_pred,cer_processed,wer_pred,wer_processed,facc_pred,facc_processed,final_score_pred,final_score_processed
nationality,0.304676,0.059578,0.852941,0.099548,0.352941,0.900452,29.536350,64.565686
Dateofbirth,0.687696,0.215266,3.839644,0.677060,0.004454,0.322940,-88.456207,38.817024
Fatherhusbandname,0.252202,0.113788,0.830000,0.388889,0.082222,0.404444,32.135262,52.466992
permanentaddress,0.310923,0.272515,1.300696,0.732715,0.000000,0.015556,13.593322,34.819292
experience,0.176803,0.145936,0.422158,0.341845,0.051111,0.144444,49.044012,52.949341
date,0.824570,0.262408,4.268182,0.936364,0.000000,0.063636,-108.246314,28.052543
presentaddress,0.315119,0.276913,1.297668,0.729767,0.000000,0.017778,13.552447,34.768859
gender,0.459750,0.008497,1.328090,0.015730,0.031461,0.984270,7.430339,69.299673
bloodgroup,0.976954,0.135271,1.578231,0.290249,0.074830,0.709751,-19.420258,55.213263
experience1,0.187324,0.152242,0.454582,0.365784,0.035556,0.124444,47.538600,51.887743


In [ ]:
metrics_df.to_csv("metrics_test_set.csv")


# WER Analysis

# Field Metric Analysis

In [225]:
field_name = 'languageknown'
df_processed[(df_processed['field_acc_processed'] == False) & (df_processed['file'] == field_name)]

,folder,file,pred_text,actual_text,processed_text,field_acc_pred,field_acc_processed
11,MIT_290,languageknown,"Tamil , English , Malayalam , Gusarati","Tamil, Malayalam, English, Gujarati","Tamil, English, Malayalam, Gusarati",False,False
133,MIT_52,languageknown,English . Telugu .,"English, Telugu",English Telugu,False,False
193,TCE_1368,languageknown,"Tarnil , English , trade .","Tamil, English, Hindi","Tarnil, English, Trade",False,False
289,TCE_1039,languageknown,"Madayalam , Manathi , Louisiana","Malayalam, Marathi, Gujarati","Madayalam, Manathi, Louisiana",False,False
326,TCE_1266,languageknown,"Gumajati , Marathi , Huischi , Malayalam","Gujarati, Marathi, Hindi, Malayalam","Gumajati, Marathi, Huischi, Malayalam",False,False
345,TCE_1174,languageknown,"Hindi's Telugu , Bengali , Gujarati","Hindi, Telugu, Bengali, Gujarati","Hindis Telugu, Bengali, Gujarati",False,False
422,TCE_1413,languageknown,"English ; Malayalam , Telugu","English, Malayalam, Telugu","English Malayalam, Telugu",False,False
440,MIT_110,languageknown,"Gujmati , Telugu .","Gujarati, Telugu","Gujmati, Telugu",False,False
460,VIT_921,languageknown,"Bengali , Tarsil , Guitarate","Bengali, Tamil, Gujarati","Bengali, Tarsil, Guitarate",False,False
480,MIT_547,languageknown,"Maratti , Kannada , Telugu .","Marathi, Kannada, Telugu","Maratti, Kannada, Telugu",False,False


### References

In [ ]:
field_name = 'referencescmob1'
df_processed[(df_processed['field_acc_processed'] == False) & (df_processed['file'] == field_name)]

In [ ]:
field_name = 'referencescmob1'
len(df_processed[(df_processed['field_acc_processed'] == False) & (df_processed['file'] == field_name)])

### Date/DOB

In [ ]:
field_name = 'date'
df_processed[(df_processed['field_acc_processed'] == False) & (df_processed['file'] == field_name)][['folder','pred_text','actual_text','processed_text']]

In [ ]:
sorted(df_processed[(df_processed['actual_text'].str.contains("-")) & (df_processed['file'] == 'Dateofbirth')]['folder'])

### Pancard

In [88]:
field_name = 'pancard'
df_processed[(df_processed['field_acc_processed'] == False) & (df_processed['file'] == field_name)]

,folder,file,pred_text,actual_text,processed_text,field_acc_pred,field_acc_processed
54,MIT_484,pancard,XIXS GO 726m.,XJXSG0726G,XIXSG0726M,False,False
95,VIT_778,pancard,VOX DB450P .,VOXDB4550P,VOXDB450P,False,False
796,MIT_35,pancard,HDE VHTO 7OOL,HDEVH6706L,HDEVHTO7OOL,False,False
836,MIT_385,pancard,ADOST 24 6H.,ADOST2463H,ADOST246H,False,False
915,MIT_174,pancard,VCJRH14937 .,VCJRH1493J,VCJRH1493T,False,False
974,MIT_488,pancard,Qumzh 374dn,QUMZH3740N,QUMZH374DN,False,False
1135,MIT_163,pancard,( BRUCIDOQV .,CBRUC1009V,BRUCIDOQV,False,False
1230,VIT_795,pancard,EPKKA3162Z ...,EPKKA3126Z,EPKKA3162Z,False,False
1447,MIT_142,pancard,TO I Q G.obby,JOIQG6906Y,TOIQGOBBY,False,False
1525,MIT_570,pancard,cehehisist .,CEHEH1313T,CEHEHISIST,False,False


In [87]:
df_processed[(df_processed['field_acc_processed'] == False) & (df_processed['file'] == field_name)].count()

folder                 62
file                   62
pred_text              62
actual_text            62
processed_text         62
field_acc_pred         62
field_acc_processed    62
dtype: int64

### Aadharcard

In [94]:
field_name = 'aadhaarcard'
df_processed[(df_processed['field_acc_processed'] == False) & (df_processed['file'] == field_name)]

,folder,file,pred_text,actual_text,processed_text,field_acc_pred,field_acc_processed
32,MIT_207,aadhaarcard,6058350000000,606000000000.0,6058350000000,False,False
52,MIT_484,aadhaarcard,151 68 1000000,751687000000,151681000000,False,False
73,MIT_101,aadhaarcard,502211800000,502000000000.0,502211800000,False,False
93,VIT_778,aadhaarcard,7874lbs000000 .0,787000000000.0,78740000000,False,False
114,MIT_587,aadhaarcard,b57.39200000,657000000000.0,5739200000,False,False
134,MIT_52,aadhaarcard,48 (583000000,482000000000.0,48583000000,False,False
174,MIT_565,aadhaarcard,"81 336,00000",813000000000.0,8133600000,False,False
232,MIT_233,aadhaarcard,223696000000,224000000000.0,223696000000,False,False
404,MIT_200,aadhaarcard,295208000000 .,295000000000.0,295208000000,False,False
441,MIT_110,aadhaarcard,"8k 44,680000bp .",864000000000.0,844680000,False,False


In [92]:
df_processed[(df_processed['field_acc_processed'] == False) & (df_processed['file'] == field_name)].count()

folder                 200
file                   200
pred_text              200
actual_text            200
processed_text         200
field_acc_pred         200
field_acc_processed    200
dtype: int64

### Contact Number

In [95]:
field_name = 'contactnumber'
print(df_processed[(df_processed['field_acc_processed'] == False) & (df_processed['file'] == field_name)].count())
df_processed[(df_processed['field_acc_processed'] == False) & (df_processed['file'] == field_name)]

folder                 233
file                   233
pred_text              233
actual_text            233
processed_text         233
field_acc_pred         233
field_acc_processed    233
dtype: int64


,folder,file,pred_text,actual_text,processed_text,field_acc_pred,field_acc_processed
15,MIT_290,contactnumber,99 28 524 27 .,9928524127,992852427,False,False
76,MIT_101,contactnumber,"9253 54,3 1 .",9253640631,92535431,False,False
117,MIT_587,contactnumber,8965IM0584 .,8965170584,89650584,False,False
137,MIT_52,contactnumber,9251995/15 .,9251995115,925199515,False,False
177,MIT_565,contactnumber,7713 4.8750 .,7713428750,771348750,False,False
254,TCE_1073,contactnumber,860-77bbbb36 .,8607766636,8607736,False,False
273,TCE_1390,contactnumber,207772976 ) .,7177219761,207772976,False,False
292,TCE_1039,contactnumber,9.89655359,9789655354,989655359,False,False
329,TCE_1266,contactnumber,82.5.2.2.237 .,8252492237,82522237,False,False
348,TCE_1174,contactnumber,94 68 68155,9468681555,946868155,False,False


### Qualification

In [109]:
field_name = 'qualification'
print(df_processed[(df_processed['field_acc_processed'] == False) & (df_processed['file'] == field_name)].count())
df_processed[(df_processed['field_acc_processed'] == False) & (df_processed['file'] == field_name)]

folder                 257
file                   257
pred_text              257
actual_text            257
processed_text         257
field_acc_pred         257
field_acc_processed    257
dtype: int64


,folder,file,pred_text,actual_text,processed_text,field_acc_pred,field_acc_processed
10,MIT_290,qualification,10th pass .,10th Pass,10th pass,False,False
112,MIT_587,qualification,Ph D .,PhD,Ph D,False,False
152,MIT_482,qualification,12th pass .,12th Pass,12th pass,False,False
172,MIT_565,qualification,10th pass .,10th Pass,10th pass,False,False
192,TCE_1368,qualification,diploma .,Diploma,diploma,False,False
250,TCE_1073,qualification,post-graduate .,Post-Graduate,post-graduate,False,False
269,TCE_1390,qualification,cloth pass .,10th Pass,cloth pass,False,False
288,TCE_1039,qualification,diploma .,Diploma,diploma,False,False
344,TCE_1174,qualification,12th pass .,12th Pass,12th pass,False,False
363,MIT_274,qualification,Diplomat .,Diploma,Diplomat,False,False


In [ ]:
keys = list(cer.keys())
val1 = [cer[k][0] for k in keys]
val2 = [cer[k][1] for k in keys]
x = np.arange(len(keys))  # Positions for keys
width = 0.35  # Width of each bar

plt.figure(figsize=(10, 4))
plt.bar(x - width/2, val1, width, label='Predicted')
plt.bar(x + width/2, val2, width, label='Processed')
plt.xticks(x, keys, rotation=90)
plt.title('CER')
plt.legend()
plt.show()

In [ ]:
keys = list(wer.keys())
val1 = [wer[k][0] for k in keys]
val2 = [wer[k][1] for k in keys]
x = np.arange(len(keys))  # Positions for keys
width = 0.35  # Width of each bar

plt.figure(figsize=(10, 4))
plt.bar(x - width/2, val1, width, label='Predicted')
plt.bar(x + width/2, val2, width, label='Processed')
plt.xticks(x, keys, rotation=90)
plt.title('WER')
plt.legend()
plt.show()

### CER/WER Analysis
- CER: Blood Group
- WER: perm/present Address, DOB/Date, Blood Group, language known, 

## Blood Group Analysis

In [ ]:
field = 'bloodgroup'
df_temp = df_processed[df_processed['file'] == field]
print(df_temp[df_temp['actual_text'] != df_temp['processed_text']][['pred_text','actual_text','processed_text']])

## Language Group Analysis

In [ ]:
pd.set_option('display.max_rows', None)
field = 'languageknown'
df_temp = df_processed[df_processed['file'] == field]
print("CER: ", char_error_rate(df_temp['actual_text'], df_temp['processed_text']))
print("WER: ", word_error_rate(df_temp['actual_text'], df_temp['processed_text']))
df_temp[df_temp['actual_text'] != df_temp['processed_text']][['pred_text','actual_text','processed_text']]

# Permanent Address

In [ ]:
pd.set_option('display.max_rows', None)
field = 'permanentaddress'
df_temp = df_processed[df_processed['file'] == field]
print("CER: ", char_error_rate(df_temp['actual_text'], df_temp['processed_text']))
print("WER: ", word_error_rate(df_temp['actual_text'], df_temp['processed_text']))
df_temp[df_temp['actual_text'] != df_temp['processed_text']][['pred_text','actual_text','processed_text']]

In [ ]:
len(df_temp[df_temp['actual_text'] != df_temp['processed_text']][['pred_text','actual_text','processed_text']])

# Date Analysis

In [ ]:
pd.set_option('display.max_rows', None)
field = 'date'
df_temp = df_processed[df_processed['file'] == field]
print("CER: ", char_error_rate(df_temp['actual_text'], df_temp['processed_text']))
print("WER: ", word_error_rate(df_temp['actual_text'], df_temp['processed_text']))
df_temp[df_temp['actual_text'] != df_temp['processed_text']][['pred_text','actual_text','processed_text']]

In [ ]:
len(df_temp[df_temp['actual_text'] != df_temp['processed_text']][['pred_text','actual_text','processed_text']])

In [ ]:
field_name = 'Dateofbirth'
df_temp = df_processed[df_processed['file'] == field_name]
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] = df_processed[['file', 'pred_text']].apply(post_processing, axis=1)
df_processed
# df_processed[df_processed['file'] == 'nationality'][['pred_text', 'processed_text']]
# char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])

In [ ]:
field_name = 'Dateofbirth'
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] =  df_processed[df_processed['file'] == field_name]['pred_text'].apply(date_specific)
# df_processed[df_processed['file'] == 'nationality'][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])

In [ ]:
field_name = 'date'
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] =  df_processed[df_processed['file'] == field_name]['pred_text'].apply(date_specific)
# df_processed[df_processed['file'] == 'nationality'][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])

In [ ]:
field_name = 'bloodgroup'
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] =  df_processed[df_processed['file'] == field_name]['pred_text'].apply(bloodgroup_specific)
# df_processed[df_processed['file'] == 'nationality'][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])

## Nationality

In [ ]:
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] =  df_processed[df_processed['file'] == 'nationality']['pred_text'].apply(remove_special_char)
df_processed[df_processed['file'] == 'nationality'][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == 'nationality']['actual_text'], df_processed[df_processed['file'] == 'nationality']['processed_text'])

In [ ]:
field_name = 'pancard'
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] =  df_processed[df_processed['file'] == field_name]['pred_text'].apply(remove_special_char)
# df_processed[df_processed['file'] == 'nationality'][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])

In [ ]:
field_name = 'maritalstatus'
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] =  df_processed[df_processed['file'] == field_name]['pred_text'].apply(remove_special_char)
# df_processed[df_processed['file'] == 'nationality'][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])

In [ ]:
field_name = 'gender'
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] =  df_processed[df_processed['file'] == field_name]['pred_text'].apply(remove_special_char)
# df_processed[df_processed['file'] == 'nationality'][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])

In [ ]:
field_name = 'contactnumber'
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] =  df_processed[df_processed['file'] == field_name]['pred_text'].apply(numeric_only)
df_processed[df_processed['file'] == field_name][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])  

In [ ]:
field_name = 'AlternateNo'
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] =  df_processed[df_processed['file'] == field_name]['pred_text'].apply(numeric_only)
df_processed[df_processed['file'] == field_name][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])  

In [ ]:
field_name = 'aadhaarcard'
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] =  df_processed[df_processed['file'] == field_name]['pred_text'].apply(numeric_only)
df_processed[df_processed['file'] == field_name][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])  

In [ ]:
field_name = 'candidatename'
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] =  df_processed[df_processed['file'] == field_name]['pred_text'].apply(char_space)
df_processed[df_processed['file'] == field_name][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])  

In [ ]:
field_name = 'Fatherhusbandname'
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] = df_processed[df_processed['file'] == field_name]['pred_text'].apply(char_space)
df_processed[df_processed['file'] == field_name][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])  

In [ ]:
field_name = 'place'
df_processed.loc[df_processed['file'] == field_name, 'processed_text'] = df_processed[df_processed['file'] == field_name]['pred_text'].apply(char_space)
df_processed[df_processed['file'] == field_name][['pred_text', 'processed_text']]
char_error_rate(df_processed[df_processed['file'] == field_name]['actual_text'], df_processed[df_processed['file'] == field_name]['processed_text'])  

In [ ]:
char_error_rate(df_processed['actual_text'], df_processed['pred_text'])

In [ ]:
word_error_rate(df_processed['actual_text'], df_processed['pred_text'])

In [ ]:
char_error_rate(df_processed['actual_text'], df_processed['processed_text'])

In [ ]:
word_error_rate(df_processed['actual_text'], df_processed['processed_text'])

In [ ]:
# No. of words
# Date: 
# Only numeric
# Only Alpha numeric
# 

# Anand OCR Model Results Analysis

In [140]:
import jellyfish
noisy_word = "sngl"
valid_labels = ['Single', 'Married']
similarities = [jellyfish.jaro_winkler_similarity(noisy_word.lower(), label.lower()) for label in valid_labels]
max_index = similarities.index(max(similarities))
valid_labels[max_index], max(similarities)

('Single', 0.8999999999999999)

In [142]:
df_temp = df_processed[df_processed['file'] == 'maritalstatus'][['pred_text', 'actual_text', 'processed_text']].copy()

In [155]:
def find_distance(texts):
    return [levenshtein_distance('Single', texts[1]), levenshtein_distance('Married', texts[1]), levenshtein_distance('UnMarried', texts[1])]

In [152]:
valid_labels = ['Single', 'Married']
def find_wink_similarity(texts):
    noisy_word = texts[1]
    similarities = [jellyfish.jaro_winkler_similarity(noisy_word.lower(), label.lower()) for label in valid_labels]
    max_index = similarities.index(max(similarities))
#     valid_labels[max_index], max(similarities)
    return max(similarities)

In [156]:
df_temp['l_dist'] = df_temp[['actual_text', 'processed_text']].apply(find_distance, axis=1)

In [153]:
df_temp['wink_sim'] = df_temp['processed_text'].apply(find_wink_similarity)

In [159]:
df_temp[df_temp['actual_text'] != df_temp['processed_text']]

,pred_text,actual_text,processed_text,l_dist,wink_sim
136,Mounied .,Married,Mounied,"[6, 3, 5]",0.000000
156,I married,Married,Imarried,"[7, 2, 3]",0.742857
291,manned .,Married,Manned,"[5, 3, 5]",0.714286
347,turn old .,Single,Turnold,"[5, 5, 7]",0.000000
367,I married,Married,Imarried,"[7, 2, 3]",0.742857
443,Sing live .,Single,Singlive,"[2, 7, 8]",0.722222
503,Niaquied .,Married,Niaquied,"[6, 4, 5]",0.722222
561,Marneed .,Married,Marneed,"[6, 2, 4]",0.714286
680,I Marnied .,Married,Imarnied,"[7, 3, 4]",0.742857
738,Mazzied .,Married,Mazzied,"[6, 2, 4]",0.714286


In [161]:
label_file = root_dir + "/" + r"Dataset\temp\train.csv"
df_act_train = pd.read_csv(label_file)

df_act_train['folder'] = df_act_train['image'].apply(lambda x: x.split("/")[2])
df_act_train['file'] = df_act_train['image'].apply(lambda x: x.split("/")[-1].replace(".jpg", ""))
df_act_train.drop('image', axis=1, inplace=True)
df_act_train.head()

,text,folder,file
0,Dayita Bakshi,MIT_1,candidatename
1,Laksh Bakshi,MIT_1,Fatherhusbandname
2,12/27/1975,MIT_1,Dateofbirth
3,Post-Graduate,MIT_1,qualification
4,Married,MIT_1,maritalstatus


In [162]:
gender_labels = df_act_train[df_act_train['file'] == 'gender']['text'].unique()
gender_labels

array(['Female', 'Male'], dtype=object)

In [163]:
marital_labels = df_act_train[df_act_train['file'] == 'maritalstatus']['text'].unique()
marital_labels

array(['Married', 'Single'], dtype=object)

In [164]:
qualification_labels = df_act_train[df_act_train['file'] == 'qualification']['text'].unique()
qualification_labels

array(['Post-Graduate', 'Graduate', 'Diploma', '10th Pass', '12th Pass',
       'PhD', 'Integrated M.tech', 'B.Tech'], dtype=object)

In [166]:
nationality_labels = df_act_train[df_act_train['file'] == 'nationality']['text'].unique()
nationality_labels

array(['Indian'], dtype=object)

In [176]:
language = df_act_train[df_act_train['file'] == 'languageknown']['text']
lang_labels = []
for i in range(len(language)):
    text = language.iloc[i]
    text = re.sub(r"[^A-Za-z0-9 ]+", "", text)
#     text = [ele.strip() for ele in text.split()]
    for ele in text.split():
        ele = ele.strip()
        if ele not in lang_labels:
            lang_labels.append(ele)
lang_labels

['Marathi',
 'English',
 'Telugu',
 'Hindi',
 'Bengali',
 'Kannada',
 'Malayalam',
 'Gujarati',
 'Tamil',
 'Kanada']

In [208]:
field_vocabulary = {}
field_vocabulary['gender'] = list(gender_labels)
field_vocabulary['maritalstatus'] = list(marital_labels)
field_vocabulary['nationality'] = list(nationality_labels)
field_vocabulary['qualification'] = list(qualification_labels)
field_vocabulary['language'] = lang_labels
field_vocabulary


{'gender': ['Female', 'Male'],
 'maritalstatus': ['Married', 'Single'],
 'nationality': ['Indian'],
 'qualification': ['Post-Graduate',
  'Graduate',
  'Diploma',
  '10th Pass',
  '12th Pass',
  'PhD',
  'Integrated M.tech',
  'B.Tech'],
 'language': ['Marathi',
  'English',
  'Telugu',
  'Hindi',
  'Bengali',
  'Kannada',
  'Malayalam',
  'Gujarati',
  'Tamil',
  'Kanada']}

In [209]:
import json
with open("field_vocabulary.json", "w") as f:
    json.dump(field_vocabulary, f)

# Load the dictionary from the JSON file


In [182]:
loaded_dict

{'gender': ['Female', 'Male'],
 'marital_status': ['Married', 'Single'],
 'nationality': ['Indian'],
 'qualification': ['Post-Graduate',
  'Graduate',
  'Diploma',
  '10th Pass',
  '12th Pass',
  'PhD',
  'Integrated M.tech',
  'B.Tech'],
 'language': ['Marathi',
  'English',
  'Telugu',
  'Hindi',
  'Bengali',
  'Kannada',
  'Malayalam',
  'Gujarati',
  'Tamil',
  'Kanada']}